In [1]:
!ls
!rm -rf *

In [2]:
!nvidia-smi

Sun Sep 21 07:58:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             10W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [ ]:
import torch
print(torch.__version__)  # Should print 2.6.0+cu124
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.device_count())  # Should print 2
print(torch.cuda.get_device_name(0))  # Should print Tesla T4
print(torch.cuda.get_device_name(1))  # Should print Tesla T4

# Test NMS
from torchvision.ops import nms

# Check installed package versions
!pip list | grep -E "torch|torchvision|torchaudio|ultralytics|roboflow|fsspec|packaging|numpy|matplotlib|scikit-learn|jupyter-client"

In [ ]:
# !pip install torch==2.5.1
# !pip install --upgrade --no-cache-dir torchvision==0.20.1+cu124 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -U ultralytics==8.2.103
!pip install -U ray[tune]

In [ ]:
import torch
import torchvision
print(torch.__version__)  # Should print 2.6.0+cu124
print(torchvision.__version__)  # Should print 0.21.0+cu124
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.device_count())  # Should print 2
print(torch.cuda.get_device_name(0))  # Should print Tesla T4
print(torch.cuda.get_device_name(1))  # Should print Tesla T4

# Test NMS
from torchvision.ops import nms
# Simple NMS test
boxes = torch.tensor([[0, 0, 10, 10], [1, 1, 11, 11], [0, 0, 5, 5]], dtype=torch.float32, device='cuda')
scores = torch.tensor([0.9, 0.8, 0.7], dtype=torch.float32, device='cuda')
keep = nms(boxes, scores, iou_threshold=0.5)
print("NMS keep indices:", keep)  # Should print tensor with indices of kept boxes

# Check installed package versions
!pip list | grep -E "torch|torchvision|torchaudio|ultralytics|roboflow|fsspec|packaging|numpy|matplotlib|scikit-learn|jupyter-client|thinc"

In [5]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6411.7/8062.4 GB disk)


In [ ]:
!yolo settings

In [7]:
from ultralytics import YOLO

from IPython.display import display, Image

In [8]:
%cd {HOME}
!yolo task=detect mode=predict model=yolov8l.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

/kaggle/working
100%|███████████████████████████████████████| 83.7M/83.7M [00:00<00:00, 214MB/s]
Ultralytics YOLOv8.2.103 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8l summary (fused): 268 layers, 43,668,288 parameters, 0 gradients, 165.2 GFLOPs

100%|████████████████████████████████████████| 104k/104k [00:00<00:00, 48.1MB/s]
image 1/1 /kaggle/working/dog.jpeg: 640x384 1 person, 1 car, 2 dogs, 2 backpacks, 60.9ms
Speed: 15.8ms preprocess, 60.9ms inference, 412.3ms postprocess per image at shape (1, 3, 640, 384)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
%cd {HOME}
Image(filename='runs/detect/predict/dog.jpeg', height=600)

In [ ]:
# model = YOLO(f'{HOME}/yolov8l.pt')
# results = model.predict(source='https://media.roboflow.com/notebooks/examples/dog.jpeg', conf=0.25)

In [ ]:
# results[0].boxes.xyxy

In [ ]:
# results[0].boxes.conf

In [ ]:
# results[0].boxes.cls

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="n12mRmQdWmrjCSKFgJMC")
project = rf.workspace("waste-detections-by-fu").project("waste-detections-c0mkz")
version = project.version(6)
dataset = version.download("yolov8")

In [ ]:
dataset.location

## Custom Training

In [ ]:
!pip list --outdated

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip list --outdated --format=json | jq -r '.[] | "\(.name)==\(.latest_version)"'

In [15]:
!cat /kaggle/working/datasets/Waste-Detections-6/data.yaml

names:
- Paper
- Plastic Bottle
- Plastic bag
- Plastic waste
nc: 4
roboflow:
  license: CC BY 4.0
  project: waste-detections-c0mkz
  url: https://universe.roboflow.com/waste-detections-by-fu/waste-detections-c0mkz/dataset/6
  version: 6
  workspace: waste-detections-by-fu
test: ../test/images
train: ../train/images
val: ../valid/images


In [31]:
!rm -rf /kaggle/working/runs/detect/train
!rm -rf /kaggle/working/runs/detect/predict
!rm -rf /kaggle/working/runs/detect/val

In [ ]:
!pip install -U ultralytics

In [ ]:
%cd {HOME}

!yolo task=detect mode=train model=yolov8l.pt data={dataset.location}/data.yaml epochs=50 batch=16 imgsz=640 plots=True device=0,1 amp=True save=True


In [ ]:
# !yolo train resume=True model={HOME}/runs/detect/train/weights/last.pt device=0,1 amp=True

In [ ]:
!yolo export model={HOME}/runs/detect/train/weights/best.pt format=coreml nms=True
# !yolo export model={HOME}/runs/detect/train3/weights/best.pt format=coreml nms=True

In [ ]:
# ! yolo train resume model={HOME}/runs/detect/train2/weights/last.pt amp=True

In [21]:
!ls

datasets  dog.jpeg  runs  yolo11n.pt  yolov8l.pt  yolov8n.pt


In [ ]:
# project.version(dataset.version).deploy(model_type="yolov8", model_path=f"{HOME}/runs/detect/train3/")

In [22]:
dataset.version

'6'

In [ ]:
!ls {HOME}/runs/detect/train/

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/results.png', width=600)

In [ ]:
%cd {HOME}
Image(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

## Validate Custom Model

In [ ]:
%cd {HOME}

!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

## Inference with Custom Model

In [ ]:
%cd {HOME}
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True

In [ ]:
import glob
from IPython.display import Image, display

for image_path in glob.glob(f'{HOME}/runs/detect/predict2/*.jpg')[:3]:
      display(Image(filename=image_path, width=600))
      print("\n")

In [ ]:
! zip -r yolov8l_recyclablewaste.zip {HOME}/runs/